In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
import opendatasets as od
import os
import re

In [ ]:
od.download('https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset')
# # sdlpkmrmkkml
# b2577bd76fc2d8e36b92620b9bd93e27

Skipping, found downloaded files in "./sentiment-analysis-dataset" (use force=True to force download)


In [ ]:
df = pd.read_csv('/content/sentiment-analysis-dataset/train.csv', encoding='latin-1')

In [ ]:
df.shape

(27481, 10)

In [ ]:
df.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

In [ ]:
df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [ ]:
# drop unnecessary columns
df.drop(['textID', 'selected_text', 'Time of Tweet', 'Age of User','Country', 'Population -2020','Land Area (Km²)','Density (P/Km²)'], axis=1, inplace=True)

In [ ]:
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [ ]:
df['sentiment'].value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [ ]:
df['sentiment'] = df['sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0})

In [ ]:
df.head()

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0


In [ ]:
df

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
...,...,...
27476,wish we could come see u on Denver husband l...,0
27477,I`ve wondered about rake to. The client has ...,0
27478,Yay good for both of you. Enjoy the break - y...,2
27479,But it was worth it ****.,2


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
df['text'] = df['text'].str.replace("@", "")
df['text'] = df['text'].str.replace("#", "")
df['text'] = df['text'].str.replace("RT", "")
df['text'] = df['text'].str.replace(":", "")
df['text'] = df['text'].str.replace(".", "")
df['text'] = df['text'].str.replace(",", "")
df['text'] = df['text'].str.replace("!", "")
df['text'] = df['text'].str.replace("?", "")
df['text'] = df['text'].str.lower()

<ipython-input-57-2b9209670e04>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(".", "")
<ipython-input-57-2b9209670e04>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace("?", "")


In [ ]:
# clean the text and ready it for vectorization and modeling
def clean_text(text):
  text = str(text)
  text = text.lower()
  text = text.split()
  text = [word for word in text if word not in stopwords.words('english')]
  text = ' '.join(text)
  return text

df['text'] = df['text'].apply(clean_text)

# vectorize the text
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(df['text']).toarray()
y = df['sentiment']

In [ ]:
 #split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# create a SVM classifier
classifier = SVC()

# fit the training data
classifier.fit(X_train, y_train)

# predict the labels of test data 
predictions = classifier.predict(X_test)

In [ ]:
pr = set(predictions)
pr

{0, 1, 2}

In [ ]:
# print the evaluation metrics
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.77      0.54      0.63      2338
           1       0.63      0.80      0.70      3371
           2       0.77      0.71      0.74      2536

    accuracy                           0.70      8245
   macro avg       0.72      0.68      0.69      8245
weighted avg       0.71      0.70      0.69      8245

[[1258  924  156]
 [ 305 2688  378]
 [  69  670 1797]]
0.696543359611886


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
filename = '/content/drive/MyDrive/finalized_Sentiment_Model.sav'
joblib.dump(classifier, filename)

['/content/drive/MyDrive/finalized_Sentiment_Model.sav']

In [ ]:
# save the vectorizer
import pickle
pickle.dump(cv, open('vectorizer.pkl', 'wb'))